## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Изучим общую информацию

In [2]:
# импортируем библиотеку pandas и прочитаем файл методом read_csv()
# выведем первые 10 строк таблицы методом head()

import pandas as pd
credit_data = pd.read_csv('/datasets/data.csv')
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# проверим информацию по таблице credit_data

credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# проверим информацию о разбросах значений количественных данных 
# по всей таблице методом describe()

credit_data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Вывод

1. Из первых 10 строк таблицы credit_data видно, что присутствуют отрицательные значения трудового стажа в днях. Данная аномалия скорее всего обсуловлена порядком расчета трудового стажа по бух. учету где из дат увольнения вычитают даты приема на работу. 
2. Также в столбце education присутствуют различия в значениях строковых литералов (верхний и нижний регистр.) 
3. Значения столбцов days_employed и total_income необходимо переформатировать с тип float на int, для упрощения расчетов и улучшения читаемости данных. Также присутствуют пропущенные данные, которые необходимо заполнить(удалить) и обработать.
4. В столбце dob_years присутствуют нулевые значения, что тоже неверно, возраст не может быть нулевым 
5. В столбце purpose данные необходимо структурировать и категоризировать для избежания дубликатов и искажения информации

Шаг 2. Предобработка данных

Обработка пропусков

In [5]:
# посмотрим количество пропущенных значений 
# и в каких столбцах значаться данные пропуски методом isna() и sum()

credit_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
# определим пропущенные значения в столбце days_employed методом isnull() 
# и выведем первые 15 строк таблицы

credit_data[credit_data['days_employed'].isnull()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Интересно, что пропущенные значения находятся как в столбце days_employed так и в total_income для одних и тех же заемщиков. Необходимо заполнить пропуски новыми значениями. Прежде всего, в столбцах total_income и days_employed, мы можем заполнить пропуски медианными значениями характерными для конкретных должностей.

In [7]:
# через цикл for пройдем по уникальным знач. столбца income_type и запишем их в виде ключей
# в словарь income_type_mean_dict, знач. ключей будут средние значения каждой категории занятости

income_type_mean_dict = {}
for i in credit_data['income_type'].unique(): 
    income_type_mean_dict[i] = credit_data[credit_data['income_type']==i]['total_income'].mean()
print(income_type_mean_dict)

{'сотрудник': 161380.26048788553, 'пенсионер': 137127.4656901654, 'компаньон': 202417.4614617771, 'госслужащий': 170898.30992266268, 'безработный': 131339.7516762103, 'предприниматель': 499163.1449470857, 'студент': 98201.62531401133, 'в декрете': 53829.13072905995}


In [8]:
# создадим функцию fill_total_income_na которая заполняет пропущенные значения в столбце 
# total_income средними значениями из словаря income_type_mean_dict

def fill_total_income_na(row):
    total_income = row['total_income']
    income_type = row['income_type']
    if pd.isnull(total_income):
        if income_type == 'сотрудник':
            return income_type_mean_dict['сотрудник']
        elif income_type == 'пенсионер':
            return income_type_mean_dict['пенсионер']
        elif income_type == 'компаньон':
            return income_type_mean_dict['компаньон']
        elif income_type == 'госслужащий':
            return income_type_mean_dict['госслужащий']
        elif income_type == 'безработный':
            return income_type_mean_dict['безработный']
        elif income_type == 'предприниматель':
            return income_type_mean_dict['предприниматель']
        elif income_type == 'студент':
            return income_type_mean_dict['студент']
        elif income_type == 'в декрете':
            return income_type_mean_dict['в декрете']
    else:
        return total_income
                                         

credit_data['total_income'] = credit_data.apply(fill_total_income_na,axis=1)

In [9]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
# через цикл for пройдем по уникальным знач. столбца days_employed и запишем их в виде ключей
# в словарь days_employed_mean_dict, знач. ключей будут средние значения каждой категории занятости

days_employed_mean_dict = {}
for i in credit_data['income_type'].unique(): 
    days_employed_mean_dict[i] = credit_data[credit_data['income_type']==i]['days_employed'].mean()
print(days_employed_mean_dict)

{'сотрудник': -2326.4992159717935, 'пенсионер': 365003.4912448612, 'компаньон': -2111.524398297732, 'госслужащий': -3399.89690169574, 'безработный': 366413.65274420456, 'предприниматель': -520.8480834953765, 'студент': -578.7515535382181, 'в декрете': -3296.7599620220594}


In [11]:
# создадим функцию fill_days_employed_na которая заполняет пропущенные значения в столбце 
# days_employed средними значениями из словаря days_employed_mean_dict

def fill_days_employed_na(new_row):
    days_employed = new_row['days_employed']
    income_type = new_row['income_type']
    if pd.isnull(days_employed):
        if income_type == 'сотрудник':
            return days_employed_mean_dict['сотрудник']
        elif income_type == 'пенсионер':
            return days_employed_mean_dict['пенсионер']
        elif income_type == 'компаньон':
            return days_employed_mean_dict['компаньон']
        elif income_type == 'госслужащий':
            return days_employed_mean_dict['госслужащий']
        elif income_type == 'безработный':
            return days_employed_mean_dict['безработный']
        elif income_type == 'предприниматель':
            return days_employed_mean_dict['предприниматель']
        elif income_type == 'студент':
            return days_employed_mean_dict['студент']
        elif income_type == 'в декрете':
            return days_employed_mean_dict['в декрете']
    else:
        return days_employed
                                         

credit_data['days_employed'] = credit_data.apply(fill_days_employed_na, axis=1)

In [12]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [13]:
credit_data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Мы заполнили пропущенные значения в столбцах days_employed и total_income средними значениями по категориям профессий заемщиков, что в целом, отражает более менее реальную картину возможных данных на месте существующих пропусков, что позволит дальше анализировать таблицу для получения более весомых результатов. 

### Замена типа данных

In [14]:
# заменим тип данных float на int в столбцах days_employed и total_income методом astype()

credit_data['days_employed'] = credit_data['days_employed'].astype('int')
credit_data['total_income'] = credit_data['total_income'].astype('int')
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [15]:
credit_data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

Заменили тип данных столбцов days_employed и total_income c float на int. Был выбран метод astype(), так как было необходимо заменить тип данных в колонках с вещественного на числовой, не создавая новых столбцов и не работая с категориальными данными типа str. 

### Обработка дубликатов

In [16]:
# удалим дубликаты из таблицы credit_data методом drop_duplicates()
# удалим появившиеся пропуски методом dropna() и уберем смещение по индексу методом reset_index()

credit_data = credit_data.drop_duplicates()
credit_data = credit_data.dropna().reset_index(drop=True)
credit_data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,-152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,-6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [17]:
# посчитаем количество дубликатов в столбце education методом value_counts()
# приведем все данные в столбце к нижнему регистру

credit_data['education'] = credit_data['education'].str.lower()
credit_data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

Данные из столбца purpose требуют категоризации в связи с наличием обобщенности целей взятия кредита. Так как комментарии, возможно, были оставлены вручную, соответственно одни и те же цели, интерпретируються по разному.
Данные из столбца education говорят о том, что больше всего кредитов берут люди со средним образованием, меньше, но тоже значительно берут кредиты люди с высшим образованием. Все остальные категории составляют незначительную часть и не влияют на выборку.

### Лемматизация

In [18]:
# импортируем библиотеку pymystem3 и запишем в переменную m
# объявим переменную new_list с пустым списком для записи значений после лемматизации

from pymystem3 import Mystem
m = Mystem()
new_list = []

# через цикл for пройдемся по столбцу purpose и применим метод m.lemmatize() к каждой строке
# добавим лемматизированные значения в список new_list

for word in credit_data['purpose']:
    lemma = m.lemmatize(word)
    for v in lemma:
        new_list.append(v)
        
# импортируем библиотеку Collections для подсчета уникальных значений

from collections import Counter
print(Counter(new_list))

Counter({' ': 33596, '\n': 21471, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


### Вывод

Можно выделить основные ключевые слова по существительным в столбце purpose: недвижимость, автомобиль, образование, жилье, свадьба

Категоризация данных

In [19]:
# создадим функцию grouped_purpose и категоризируем данные в столбце purpose 
# исходя из уникальных значений

def grouped_purpose(row):
    purpose = row['purpose']
    if 'недвиж' in purpose:
        return 'недвижимость'
    elif 'авто' in purpose:
        return 'автомобиль'
    elif 'образов' in purpose:
        return 'образование'
    elif 'жил' in purpose:
        return 'жилье'
    elif 'свад' in purpose:
        return 'свадьба'
    return 'другие цели'


credit_data['purpose'] = credit_data.apply(grouped_purpose, axis=1)

In [20]:
credit_data['purpose'].value_counts()

недвижимость    6353
жилье           4461
автомобиль      4308
образование     4014
свадьба         2335
Name: purpose, dtype: int64

Больше всего кредитов берут на недвижмость (жилую или коммерческую), примерно одинаковые показатели у целей "автомобиль" и "образование", и меньше всего на свадьбу.

In [21]:
# создадим функцию children_count для категоризации по количеству детей столбца children

def children_count(row):
    children = row['children']
    if children == 0:
        return 'нет детей'
    elif children == 1:
        return 'один ребенок'
    elif children > 1:
        return 'многодетная семья'

credit_data['children_categorized'] = credit_data.apply(children_count, axis=1)

In [22]:
credit_data['children_categorized'].value_counts()

нет детей            14107
один ребенок          4809
многодетная семья     2508
Name: children_categorized, dtype: int64

На основании категоризации по количество детей, можно сделать промежуточный вывод, что кредиты в основном берут люди у которых нет детей, продолжим исследование

In [23]:
# определим максимальные, минимальные и средние значеня столбца total_income методами min(), 
# max() и mean(), чтобы взять пороговые значения и категоризировать заемщиков по благосостоянию

credit_data['total_income'].min()

20667

In [24]:
credit_data['total_income'].max()

2265604

In [25]:
credit_data['total_income'].mean()

167421.57826836198

In [26]:
# создадим функцию total_income_categorized, которая категоризирует столбец total_income 
# по уровню дохода
# возьмем диапазон 25% от пороговых значений


def total_income_categorized(row):
    total_income = row['total_income']
    if credit_data['total_income'].min() <= total_income <= (credit_data['total_income'].mean()) - (credit_data['total_income'].mean() * 0.25):
        return 'низкий доход'
    elif (credit_data['total_income'].mean()) - (credit_data['total_income'].mean() * 0.25) <= total_income <= (
        credit_data['total_income'].mean()) + (
        credit_data['total_income'].mean() * 0.25
    ):
        return 'средний доход'
    elif credit_data['total_income'].max() >= total_income >= (credit_data['total_income'].mean()) + (credit_data['total_income'].mean() * 0.25):
        return 'высокий доход'
    
credit_data['total_income_categorized'] = credit_data.apply(total_income_categorized, axis=1)

In [27]:
credit_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_categorized,total_income_categorized
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье,один ребенок,высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,один ребенок,низкий доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье,нет детей,средний доход
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетная семья,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,средний доход


In [28]:
credit_data['total_income_categorized'].value_counts()

средний доход    9504
низкий доход     7421
высокий доход    4546
Name: total_income_categorized, dtype: int64

Больше всего заемщиков со средним доходом. Затем идут заемщики с низким доходом и высоким.

In [29]:
# посмотрим, как от доходов зависит цель кредита, сгруппируем методом groupdby()
# столбцы total_income и purpose и применим метод value_counts()

credit_data.groupby('purpose')['total_income_categorized'].value_counts()

purpose       total_income_categorized
автомобиль    средний доход               1893
              низкий доход                1476
              высокий доход                939
жилье         средний доход               1957
              низкий доход                1544
              высокий доход                960
недвижимость  средний доход               2844
              низкий доход                2166
              высокий доход               1343
образование   средний доход               1797
              низкий доход                1404
              высокий доход                813
свадьба       средний доход               1013
              низкий доход                 831
              высокий доход                491
Name: total_income_categorized, dtype: int64

Можно заметить, что больше всего кредитов на авто берут люди со средним достатком, они вероятнее всего могут позволить купить себе машину сразу, но трата больших сумм целиком нецелесообразна, поэтому они предпочитают взять всю сумму или часть в кредит, чтобы снизить нагрузу на бюджет. 

На покупку жилья также больше всего кредитов берут люди со средним доходом.

Интересно, что люди с низким доходом практически как и со средним доходом больше всего берут кредитов на покупку недвижимости, вероятно, это жилая недвижимость, потому что такую крупную, разовую покупку они просто не могут себе позволить.

Тоже самое касается и образования, здесь кредит, скорее всего, обусловлен желанием развиваться и увеличить свой доход, при этом не тратя крупную сумму денег единоразово. По сути, некоторая инвестиция в собственный рост.

In [30]:
# посмотрим какая категория по семейному положению больше всего берет кредитов методом value_counts()

credit_data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [31]:
# сделаем словари по разным категориям и наличию (отсутствию) задолженности

income_debt = credit_data[['income_type', 'total_income_categorized', 'debt']]
income_debt.head(15)

,income_type,total_income_categorized,debt
0,сотрудник,высокий доход,0
1,сотрудник,низкий доход,0
2,сотрудник,средний доход,0
3,сотрудник,высокий доход,0
4,пенсионер,средний доход,0
5,компаньон,высокий доход,0
6,компаньон,высокий доход,0
7,сотрудник,средний доход,0
8,сотрудник,низкий доход,0
9,сотрудник,средний доход,0


In [32]:
# тоже самое проделаем и для семейного статуса

family_debt = credit_data[['family_status', 'children_categorized', 'debt']]
family_debt.head(15)

,family_status,children_categorized,debt
0,женат / замужем,один ребенок,0
1,женат / замужем,один ребенок,0
2,женат / замужем,нет детей,0
3,женат / замужем,многодетная семья,0
4,гражданский брак,нет детей,0
5,гражданский брак,нет детей,0
6,женат / замужем,нет детей,0
7,женат / замужем,нет детей,0
8,гражданский брак,многодетная семья,0
9,женат / замужем,нет детей,0


In [33]:
# сделаем словарь по целям заема и наличию или отсутствию задолженностей

purpose_debt = credit_data[['purpose', 'debt']]
purpose_debt.head(15)

,purpose,debt
0,жилье,0
1,автомобиль,0
2,жилье,0
3,образование,0
4,свадьба,0
5,жилье,0
6,жилье,0
7,образование,0
8,свадьба,0
9,жилье,0


Вывод

Мы категоризировали данные в столбцах children, total_income, purpose. Сделали промежуточные выводы, где увидели, что: 

Больше всего кредитов берут на недвижмость (жилую или коммерческую), примерно одинаковые показатели у целей "автомобиль" и "образование", и меньше всего на свадьбу.

Кредиты в основном берут люди у которых нет детей

Больше всего заемщиков с низким доходом. Примерно одинаковое количество со средними и высокими доходами.

Среди заемщиков, большое количество людей женаты или замужем, примерно одинаковое количество свободны или состоят в гражданском браке

###  Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
# создадим функцию children_credit которая будет принимать значения столбца children и debt
# функция будт возвращать категории людей которые (не)имеют детей и (не)возвращают кредит в срок
# результаты запишем в новый столбец children_depences

def children_credit(row):
    children = row['children_categorized']
    debt = row['debt']
    if children == 'один ребенок':
        if debt == 1:
            return 'должник с одним ребенком'
        else:
            return 'один ребенок и недолжник'
    elif children == 'нет детей':
        if debt == 1:
            return 'должник без детей'
        else:
            return 'без детей и недолжник'
    elif children == 'многодетная семья':
        if debt == 1:
            return 'должник с детьми'
        else:
            return 'много детей и недолжник'


credit_data['children_depences'] = credit_data.apply(children_credit, axis=1)
credit_data['children_depences'].value_counts()

без детей и недолжник       13044
один ребенок и недолжник     4365
много детей и недолжник      2275
должник без детей            1063
должник с одним ребенком      444
должник с детьми              233
Name: children_depences, dtype: int64

In [35]:
# проверим процентное соотношение (не)должников (с детьми или без детей) 
# по отношению к общему количеству людей

percentage_children_credit = credit_data['children_depences'].value_counts() / len(credit_data['children_depences'])
for i in percentage_children_credit:
    print('{:.2%}'.format(i))

60.75%
20.33%
10.60%
4.95%
2.07%
1.09%


In [36]:
# составим сводную таблицу методом pivot_table по категории children и посмотрим результат

pivot_children = credit_data.pivot_table(index='children_categorized', columns='debt', aggfunc='size', fill_value=0)
pivot_children.head()

debt,0,1
children_categorized,,
многодетная семья,2275,233
нет детей,13044,1063
один ребенок,4365,444


In [37]:
# определим долю должников в каждой категории children

print('Доля должников в многодетных семьях - {:.2%}'.format(233 / (2275 + 233)))
print('Доля должников без детей - {:.2%}'.format(1063 / (13044 + 1063)))
print('Доля должников с одним ребенком - {:.2%}'.format(444 / (4365 + 444)))

Доля должников в многодетных семьях - 9.29%
Доля должников без детей - 7.54%
Доля должников с одним ребенком - 9.23%


### Вывод

На основании расчетов можно сделать вывод:

1. Доля должников в многодетных семьях - 9.29%
2. Доля должников без детей - 7.54%
3. Доля должников с одним ребенком - 9.23%

Данные говорят о том, что меньше всего должников в категории "без детей" 7.54% и больше всего в многодетных семьях 9.29% и почти столько же у кого хотя бы один ребенок 9.23%. 
Соответственно, можем наблюдать прямую зависимость - чем больше детей, тем больше вероятность просрочки. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
# создадим функцию marriage_status которая будет принимать значения столбца family_status и debt
# функция будет возвращать категории людей которые (не)женаты/замужем (не)возвращают кредит в срок
# результаты запишем в новый столбец marriage_depences

def marriage_status(new_row):
    family_status = new_row['family_status']
    debt = new_row['debt']
    if family_status == 'женат / замужем':
        if debt == 0:
            return 'женат/замужем и без просрочек'
        else:
            return 'женат/замужем и были просрочки'        
    elif family_status == 'гражданский брак':
        if debt == 0:
            return 'в гражданском браке и нет просрочек'
        else:
            return 'в гражданском браке и есть просрочки'
    elif not family_status == 'женат / замужем' and not family_status == 'гражданский брак':
        if debt == 0:
            return 'свободен и без просрочек'
        else:
            return 'свободен и есть просрочки'
        
        
credit_data['marriage_depences'] = credit_data.apply(marriage_status, axis=1)
credit_data['marriage_depences'].value_counts()

женат/замужем и без просрочек           11413
свободен и без просрочек                 4542
в гражданском браке и нет просрочек      3775
женат/замужем и были просрочки            931
свободен и есть просрочки                 422
в гражданском браке и есть просрочки      388
Name: marriage_depences, dtype: int64

In [39]:
percentage_marriage_credit = credit_data['marriage_depences'].value_counts() / len(credit_data['marriage_depences'])
for v in percentage_marriage_credit:
    print('{:.2%}'.format(v))

53.16%
21.15%
17.58%
4.34%
1.97%
1.81%


In [40]:
# составим сводную таблицу методом pivot_table по категории family_status и посмотрим результат

pivot_marriage = credit_data.pivot_table(index='family_status', columns='debt',aggfunc='size', fill_value=0)
pivot_marriage.head()

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3775,388
женат / замужем,11413,931


In [49]:
# определим долю должников в каждой категории family

print('Доля должников в категории не женат/не замужем - {:.2%}'.format(274 / (2536 + 274)))
print('Доля должников в разводе - {:.2%}'.format(85 / (1110 + 85)))
print('Доля должников вдовец/вдова - {:.2%}'.format(63 / (896 + 63)))
print('Доля должников в гражданском браке - {:.2%}'.format(388 / (3775 + 388)))
print('Доля должников в категории женат/замужем - {:.2%}'.format(931 / (11413 + 931)))

Доля должников в категории не женат/не замужем - 9.75%
Доля должников в разводе - 7.11%
Доля должников вдовец/вдова - 6.57%
Доля должников в гражданском браке - 9.32%
Доля должников в категории женат/замужем - 7.54%


### Вывод

На основании расчетов можно сделать вывод:

1. Доля должников в категории не женат/не замужем - 9.75%
2. Доля должников в разводе- 7.11%
3. Доля должников вдовец/вдова- 6.57%
4. Доля должников в гражданском браке- 9.32%
5. Доля должников в категории женат/замужем - 7.54%

Данные говорят о том, что больше всего должников в категории не женат/не замужем 9.75% и в гражданском браке 9.32%. Меньше всего у вдовцов/вдов 6.57%. По сути, наблюдается некоторая зависимость между семейным положением, поскольку люди, которые имеют мужа или жену, чаще возвращают кредиты без просрочек 7.54 чем те, кто не обременем семейным положением(гражданский брак тоже относится к категории не женат/не замужем), и по всему видимо менее ответственнен. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [50]:
# создадим функцию income_depences которая будет принимать значения столбца total_income и debt
# функция будет возвращать категории людей которые (не)имеют детей и (не)возвращают кредит в срок
# результаты запишем в новый столбец income depences

def income_depences(row):
    total_income_categorized = row['total_income_categorized']
    debt = row['debt']
    if total_income_categorized == 'высокий доход':
        if debt == 0:
            return 'люди с высоким доходом и без просрочек'
        else:
            return 'люди с высоким доходом и с просрочками'
    elif total_income_categorized == 'средний доход':
        if debt == 0:
            return 'люди со средним доходом и без просрочек'
        else:
            return 'люди со средним доходом и с просрочками'
    elif total_income_categorized == 'низкий доход':
        if debt == 0:
            return 'люди с низким доходом без просрочек'
        else:
            return 'люди с низким доходом и с просрочками'

credit_data['income_depences'] = credit_data.apply(income_depences, axis=1)
credit_data['income_depences'].value_counts()

люди со средним доходом и без просрочек    8694
люди с низким доходом без просрочек        6810
люди с высоким доходом и без просрочек     4226
люди со средним доходом и с просрочками     810
люди с низким доходом и с просрочками       611
люди с высоким доходом и с просрочками      320
Name: income_depences, dtype: int64

In [51]:
percentage_children_credit = credit_data['income_depences'].value_counts() / len(credit_data['income_depences'])
for с in percentage_children_credit:
    print('{:.2%}'.format(с))

40.49%
31.72%
19.68%
3.77%
2.85%
1.49%


In [52]:
# составим сводную таблицу методом pivot_table по категории total_income_categorized и посмотрим результат

pivot_income = credit_data.pivot_table(index='total_income_categorized', columns='debt',aggfunc='size', fill_value=0)
pivot_income.head()

debt,0,1
total_income_categorized,,
высокий доход,4226,320
низкий доход,6810,611
средний доход,8694,810


In [53]:
# определим долю должников в каждой категории income

print('Доля должников с высоким доходом - {:.2%}'.format(320 / (4226 + 320)))
print('Доля должников с низким доходом - {:.2%}'.format(611 / (6810 + 611)))
print('Доля должников с средним доход - {:.2%}'.format(810 / (8694 + 810)))

Доля должников с высоким доходом - 7.04%
Доля должников с низким доходом - 8.23%
Доля должников с средним доход - 8.52%


### Вывод

На основании расчетов можно сделать вывод:

1. Доля должников с высоким доходом - 7.04%
2. Доля должников с низким доходом - 8.23%
3. Доля должников с средним доход - 8.52%

Данные говорят о том, что больше всего должников в категории со средним доходом 8.52%, и почти столько же с низким доходом 8.23%. Меньше всего должников с высоким доходом 7.04%. Данная зависимость говорит о том, что чем больше доход, тем меньше вероятность просрочки по кредиту. 

- Как разные цели кредита влияют на его возврат в срок?

In [54]:
# создадим функцию purpose_depences которая будет принимать значения столбца purpose и debt
# функция будет возвращать категории людей которые в зависимости от целей либо возвращают 
# кредит в срок либо нет
# результаты запишем в новый столбец purpose depences

def purpose_depences(row):
    purpose = row['purpose']
    debt = row['debt']
    if purpose == 'недвижимость':
        if debt == 1:
            return 'покупка недвижимости и должник'
        else:
            return 'покупка недвижимости и недолжник'
    if purpose == 'жилье':
        if debt == 1:
            return 'покупка жилья и должник '
        else:
            return 'покупка жилья и недолжник'
    if purpose == 'автомобиль':
        if debt == 1:
            return 'покупка авто и должник'
        else:
            return 'покупка авто и недолжник'
    if purpose == 'образование':
        if debt == 1:
            return 'оплата образования и должник'
        else:
            return 'оплата образования и недолжник'
    if purpose == 'свадьба':
        if debt == 1:
            return 'оплата свадьбы и должник'
        else:
            return 'оплата свадьбы и недолжник'
        
        
credit_data['purpose_depences'] = credit_data.apply(purpose_depences, axis=1)
credit_data['purpose_depences'].value_counts()

покупка недвижимости и недолжник    5879
покупка жилья и недолжник           4153
покупка авто и недолжник            3905
оплата образования и недолжник      3644
оплата свадьбы и недолжник          2149
покупка недвижимости и должник       474
покупка авто и должник               403
оплата образования и должник         370
покупка жилья и должник              308
оплата свадьбы и должник             186
Name: purpose_depences, dtype: int64

In [55]:
percentage_children_credit = credit_data['purpose_depences'].value_counts() / len(credit_data['purpose_depences'])
for e in percentage_children_credit:
    print('{:.2%}'.format(e))

27.38%
19.34%
18.19%
16.97%
10.01%
2.21%
1.88%
1.72%
1.43%
0.87%


In [56]:
# составим сводную таблицу методом pivot_table по категории purpose и посмотрим результат

pivot_purpose = credit_data.pivot_table(index='purpose', columns='debt',aggfunc='size', fill_value=0)
pivot_purpose.head()

debt,0,1
purpose,,
автомобиль,3905,403
жилье,4153,308
недвижимость,5879,474
образование,3644,370
свадьба,2149,186


In [58]:
# определим долю должников в каждой категории purpose

print('Доля должников в категории автомобиль - {:.2%}'.format(403 / (3905 + 403)))
print('Доля должников в категории жилье - {:.2%}'.format(308 / (4153 + 308)))
print('Доля должников в категории недвижимость - {:.2%}'.format(474 / (5879 + 474)))
print('Доля должников в категории образование - {:.2%}'.format(370 / (3644 + 370)))
print('Доля должников в категории свадьба- {:.2%}'.format(186 / (2149 + 186)))

Доля должников в категории автомобиль - 9.35%
Доля должников в категории жилье - 6.90%
Доля должников в категории недвижимость - 7.46%
Доля должников в категории образование - 9.22%
Доля должников в категории свадьба- 7.97%


### Вывод

На основании расчетов можно сделать вывод:

1. Доля должников в категории автомобиль - 9.35%
2. Доля должников в категории жилье - 6.90%
3. Доля должников в категории недвижимость - 7.46%
4. Доля должников в категории образование - 9.22%
5. Доля должников в категории свадьба- 7.97%

Данные говорят о том, что больше всего должников в категории автомобиль(9.35%) и образование(9.22%). Меньше всего в категории жилье (6.90%) и недвижимость (7.46%). Видимо есть зависимость между объемом занимаемых средств и возвращаемостью кредита, чем больше покупка и сумма кредита тем меньше вероятность просрочки. 

### Общий вывод

В рамках данного исследования была проведена работа по анализу данных заемщиков, в ходе которой были очищены или заменены ошибочные или пропущенные данные, для подведения более точных выводов и итогов. В анализе использовались такие инструменты как получение общей, всесторонней информации по представленным данным, для верхнеуровневого понимания общей картины, вычисления средней величины дохода и трудового стажа для замены пропущенных значений в данных категориях. 

Были удалены дубликаты, которые так или иначе не влияют на общие данные в целом, но портят читабельность таблицы. Категория "образование" была приведена к общему, нижнему регистру, так как из за синтаксического различия присутствовало искажение данных. 

Затем, была проведена категоризация и обобщение данных по столбцах total_income, purpose, children для повышения читабельности таблицы и упрощения дальнейших проводимых расчетов для выяснения % задолжников и недолжников по отношению к общему количеству заемщиков. 

С помощью функций были созданы дополнительные столбцы, которые показывали какое количество людей по определенным категориям и признакам (количество детей, уровень дохода, семейное положение, цели взятия кредита) ранее имели задолженность по кредитам или нет. А также, какие категории больше всего представлена в данных таблицы credit_data.

На основе результатов анализа можно сделать общие выводы:

1. Данные говорят о том, что меньше всего должников в категории "без детей" 7.54% и больше всего в многодетных семьях 9.29% и почти столько же у кого хотя бы один ребенок 9.23%. Соответственно, можем наблюдать прямую зависимость - чем больше детей, тем больше вероятность просрочки.
2. Данные говорят о том, что больше всего должников в категории не женат/не замужем 9.75% и в гражданском браке 9.32%. Меньше всего у вдовцов/вдов 6.57%. По сути, наблюдается некоторая зависимость между семейным положением, поскольку люди, которые имеют мужа или жену, чаще возвращают кредиты без просрочек 7.54 чем те, кто не обременем семейным положением(гражданский брак тоже относится к категории не женат/не замужем), и по всему видимо менее ответственнен.
3. Данные говорят о том, что больше всего должников в категории со средним доходом 8.52%, и почти столько же с низким доходом 8.23%. Меньше всего должников с высоким доходом 7.04%. Данная зависимость говорит о том, что чем больше доход, тем меньше вероятность просрочки по кредиту.
4. Данные говорят о том, что больше всего должников в категории автомобиль(9.35%) и образование(9.22%). Меньше всего в категории жилье (6.90%) и недвижимость (7.46%). Видимо есть зависимость между объемом занимаемых средств и возвращаемостью кредита, чем больше покупка и сумма кредита тем меньше вероятность просрочки.